# Teaching ML algorithms how to add, divide, modulo, and use exponents.

### Intro
This work was inspired by blog post by Mario Filho where he was training XgBoost on data for the 4 basic mathematical operations of add, subtract, multiply, and divde. The blog post can be found here:
https://www.mariofilho.com/can-gradient-boosting-learn-simple-arithmetic/

I was interested by his initial findings and was curious to see how other machine learning algorithms would perform. How hard is it for advanced algorithms to learn other basic mathematical operations like modulo and exponents.

### Approach

1. Generate a dataset of 1,000 rows of number pairs ranging between zero and ten. Produce a target variable for each mathematical operation by applying it to each row of number pairs in the feature vector.
2. Create a dictionary with various ML algorithm instantiations. This notebook currently tests the following algorithms:
- Linear Regression
- Ridge Regression
- Ada Boost Regressor
- Random Forest Regressor
- Gaussian Process Regressor
- XgBoost Regressor
- Multi-layer Perceptron regressor

3. Split the data with a train-test split of 33% test, 67% train.
4. Loop over the operations and over the models. Train each model for an operation, predict the target variable on the test feature vector and then compare the prediction of the test target variable with the orginal value. Calculate the RMSE and the R2 score of the prediction.
5. Identify the models that performed the best for each metric for each of the four mathematical operations.


### Results
Results for models built with a dataset containing 1,000 rows

| Operation   | Metric   | Model                      |
|:------------|:---------|:---------------------------|
| Addition    | RMSE     | Linear Regression          |
| Addition    | R2_score | Linear Regression          |
| Division    | RMSE     | XgBoost Regressor          |
| Division    | R2_score | XgBoost Regressor          |
| Modulo      | RMSE     | Random Forest Regressor    |
| Modulo      | R2_score | Random Forest Regressor    |
| Power       | RMSE     | Gaussian Process Regressor |
| Power       | R2_score | Gaussian Process Regressor |

### Next steps:
- Expand the feature vector space to include 0 and negative numbers.
- Test more operations
- Add more sklearn models (deep learning?)
- Metric results as a function of dataset size (n = 10, 100, 1000, 1000) Make a log log plot of metrics vs. dataset size with different lines for each function.

In [1]:
from operator import add, truediv, mod, pow

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

from xgboost import XGBRegressor

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# set numpy random seed
np.random.seed(0)

## Creating our feature vector and target variable

Create a FV with 1,000 rows and two features. These two features are the numbers that will receive the mathematical operation.

Then we create 4 different target variables based on the mathematical operation we are trying to reproduce via ML.

In [4]:
# create X feature vector
X = 10*np.random.rand(10000, 2)

In [5]:
# create target variable for adding
y_add = np.fromiter(map(add, [x[0] for x in X],[x[1] for x in X]), dtype=float)

# check the creation of the target variable
for idx, x in enumerate(X):
    assert(x[0]+x[1]==y_add[idx])

In [6]:
# create target variable for division
y_div = np.fromiter(map(truediv, [x[0] for x in X],[x[1] for x in X]), dtype=float)

# check the creation of the target variable
for idx, x in enumerate(X):
    assert(x[0]/x[1]==y_div[idx])

In [7]:
# create target variable for mod
y_mod = np.fromiter(map(mod, [x[0] for x in X],[x[1] for x in X]), dtype=float)

# check the creation of the target variable
for idx, x in enumerate(X):
    assert(x[0]%x[1]==y_mod[idx])

In [8]:
# create target variable for powers
y_pow = np.fromiter(map(pow, [x[0] for x in X],[x[1] for x in X]), dtype=float)

# check the creation of the target variable
for idx, x in enumerate(X):
    assert(x[0]**x[1]==y_pow[idx])

# Operations and Model dictionaries

In [9]:
# create a dictionary of the 4 types mathetical operations the algorithms will learn
op_target_dict = {"Addition": y_add,
                  "Division": y_div,
                  "Modulo": y_mod,
                  "Power": y_pow,}

In [10]:
model_dict = {"Linear Regression": LinearRegression(),
              "Ridge Regression": Ridge(random_state=0),
              "Ada Boost Regressor": AdaBoostRegressor(random_state=0),
              "Random Forest Regressor": RandomForestRegressor(random_state=0),
              "Gaussian Process Regressor": GaussianProcessRegressor(random_state=0),
              "XgBoost Regressor": XGBRegressor(random_state=0),
              "Multi-layer Perceptron regressor": MLPRegressor(max_iter=1000, learning_rate_init=0.01,
                                                               learning_rate='adaptive', random_state=0)
             }

# Training and Testing
 - Train-test split the data set with a test size of 33%.
 - Loop through every operation in the op_target_dict.
 - Loop through every model in the model_dict.
 - Train the model
 - Score the test feature vector
 - Calculated the RMSE and the r2 score for the test target variable.
 - Save results to a dataframe.
 
Note: The StandardScalar was considered for use, since that is the best practice for certain algorithms such as the multi-layer percepton regressor. However, it was not considered, so as to no muddy the relationships between the input values and the output target from the mathematical operations. It was unclear if the standardscaler would decrease the effectiveness of other algorithms.

In [11]:
outputs = []

for operation, y_ in op_target_dict.items():
    #Standard Scalar optional here
    #X = StandardScaler().fit_transform(X)
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y_, test_size=0.33, random_state=42)
    
    for model_name, model in model_dict.items():
        print(f"Training model {model_name} on operation {operation}")
        model_fit = model.fit(X_train_, y_train_, )
        train_score_ = model_fit.score(X_train_, y_train_)
        y_pred_ = model_fit.predict(X_test_)
        rmse_ = mean_squared_error(y_test_, y_pred_, squared=False)
        r2_ = r2_score(y_test_, y_pred_)
        outputs.append(pd.DataFrame(data={"Operation": [operation],"Model": [model_name],
                                          "train_score": [train_score_], "RMSE": [rmse_],
                                          "R2_score": [r2_],
                                          "abs_R2_score-1": [abs(r2_-1)],}))
df_results = pd.concat(outputs)

Training model Linear Regression on operation Addition
Training model Ridge Regression on operation Addition
Training model Ada Boost Regressor on operation Addition
Training model Random Forest Regressor on operation Addition
Training model Gaussian Process Regressor on operation Addition
Training model XgBoost Regressor on operation Addition
Training model Multi-layer Perceptron regressor on operation Addition
Training model Linear Regression on operation Division
Training model Ridge Regression on operation Division
Training model Ada Boost Regressor on operation Division
Training model Random Forest Regressor on operation Division
Training model Gaussian Process Regressor on operation Division
Training model XgBoost Regressor on operation Division
Training model Multi-layer Perceptron regressor on operation Division
Training model Linear Regression on operation Modulo
Training model Ridge Regression on operation Modulo
Training model Ada Boost Regressor on operation Modulo
Training

/Users/u398323/miniconda3/envs/bryan-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
# identify the best model for each operation by metric
best_model_outputs = []

for op_ in op_target_dict.keys():
    
    best_model_outputs.append(pd.DataFrame(data={"Operation": [op_],
                                                 "Metric": ['RMSE'],
                    "Value": [df_results[df_results.Operation==op_].sort_values('RMSE').RMSE.values[0]],
                    "Model": [df_results[df_results.Operation==op_].sort_values('RMSE').Model.values[0]]
                                                }))
    best_model_outputs.append(pd.DataFrame(data={"Operation": [op_],
                                             "Metric": ['R2_score'],
        "Value": [df_results[df_results.Operation==op_].sort_values('abs_R2_score-1').R2_score.values[0]],
                    "Model": [df_results[df_results.Operation==op_].sort_values('abs_R2_score-1').Model.values[0]],
                                            }))

df_best_models = pd.concat(best_model_outputs)

In [13]:
# show the best models for operations by metrics
display(df_best_models.sort_values(['Operation']))

,Operation,Metric,Value,Model
0,Addition,RMSE,0.000,Linear Regression
0,Addition,R2_score,1.000,Linear Regression
0,Division,RMSE,144.426,XgBoost Regressor
0,Division,R2_score,0.408,XgBoost Regressor
0,Modulo,RMSE,0.496,Gaussian Process Regressor
0,Modulo,R2_score,0.945,Gaussian Process Regressor
0,Power,RMSE,23170.819,Gaussian Process Regressor
0,Power,R2_score,1.000,Gaussian Process Regressor


In [14]:
# tabulate best model list for markdown format
print(df_best_models[['Operation', 'Metric', 'Model']].to_markdown(index=False))

| Operation   | Metric   | Model                      |
|:------------|:---------|:---------------------------|
| Addition    | RMSE     | Linear Regression          |
| Addition    | R2_score | Linear Regression          |
| Division    | RMSE     | XgBoost Regressor          |
| Division    | R2_score | XgBoost Regressor          |
| Modulo      | RMSE     | Gaussian Process Regressor |
| Modulo      | R2_score | Gaussian Process Regressor |
| Power       | RMSE     | Gaussian Process Regressor |
| Power       | R2_score | Gaussian Process Regressor |


In [15]:
# final results for all operations and models
display(df_results)

,Operation,Model,train_score,RMSE,R2_score,abs_R2_score-1
0,Addition,Linear Regression,1.000,0.000,1.000,0.000
0,Addition,Ridge Regression,1.000,0.000,1.000,0.000
0,Addition,Ada Boost Regressor,0.964,0.795,0.962,0.038
0,Addition,Random Forest Regressor,1.000,0.049,1.000,0.000
0,Addition,Gaussian Process Regressor,1.000,0.000,1.000,0.000
0,Addition,XgBoost Regressor,1.000,0.110,0.999,0.001
0,Addition,Multi-layer Perceptron regressor,1.000,0.017,1.000,0.000
0,Division,Linear Regression,0.021,187.329,0.004,0.996
0,Division,Ridge Regression,0.021,187.329,0.004,0.996
0,Division,Ada Boost Regressor,0.932,154.592,0.322,0.678
